## Casos ativos infinitos? O que pode estar acontecendo?

Percebemos valores anormais para os casos relatados da COVID-19. Algumas células de casos ativos estão com valores infinitos. Isso pode ser por:

[  ] A taxa de notificação estar zerada. *Por que isso aconteceria?*

[  ] O cálculo desconsidera algum caso de borda.

[  ] Outros.


In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999

import warnings
warnings.filterwarnings('ignore')

In [2]:
from src.loader.utils import get_config

config = get_config("https://raw.githubusercontent.com/ImpulsoGov/simulacovid/master/src/configs/config.yaml")

In [3]:
df = pd.read_csv("http://45.55.43.231:7000/br/cities/cases/full")

In [4]:
df.head(10)

,active_cases,city_id,city_name,confirmed_cases,daily_cases,daily_cases_growth,daily_cases_mavg_1mi,deaths,deaths_mavg,epidemiological_week,estimated_cases,estimated_population_2019,expected_mortality,health_region_id,health_region_name,infectious_period_cases,is_last,is_repeated,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_updated,new_deaths,new_deaths_growth,new_deaths_mavg_1mi,notification_rate,order_for_place,place_type,state_id,state_name,state_num_id,total_estimated_cases,data_last_refreshed
0,NaN,1100015,Alta Floresta D'Oeste,1.0,1.0,estabilizando,NaN,0.0,0.066667,18,NaN,22945.0,0.005314,11005,Zona da Mata,1.0,False,False,4.35825,2020-05-02,0.0,2020-05-02,0.0,estabilizando,NaN,NaN,1,city,RO,Rondônia,11,NaN,2020-08-14 02:17:21
1,NaN,1100015,Alta Floresta D'Oeste,1.0,0.0,estabilizando,NaN,0.0,0.066667,19,NaN,22945.0,0.005314,11005,Zona da Mata,1.0,False,False,4.35825,2020-05-03,0.0,2020-05-03,0.0,estabilizando,NaN,NaN,2,city,RO,Rondônia,11,NaN,2020-08-14 02:17:21
2,NaN,1100015,Alta Floresta D'Oeste,1.0,0.0,estabilizando,NaN,0.0,0.066667,19,NaN,22945.0,0.005314,11005,Zona da Mata,1.0,False,False,4.35825,2020-05-04,0.0,2020-05-04,0.0,estabilizando,NaN,NaN,3,city,RO,Rondônia,11,NaN,2020-08-14 02:17:21
3,NaN,1100015,Alta Floresta D'Oeste,1.0,0.0,estabilizando,NaN,0.0,0.066667,19,NaN,22945.0,0.005314,11005,Zona da Mata,1.0,False,False,4.35825,2020-05-05,0.0,2020-05-05,0.0,estabilizando,NaN,NaN,4,city,RO,Rondônia,11,NaN,2020-08-14 02:17:21
4,NaN,1100015,Alta Floresta D'Oeste,1.0,0.0,estabilizando,NaN,0.0,0.066667,19,NaN,22945.0,0.005314,11005,Zona da Mata,1.0,False,False,4.35825,2020-05-06,0.0,2020-05-06,0.0,estabilizando,NaN,NaN,5,city,RO,Rondônia,11,NaN,2020-08-14 02:17:21
5,NaN,1100015,Alta Floresta D'Oeste,1.0,0.0,estabilizando,NaN,0.0,0.066667,19,NaN,22945.0,0.005314,11005,Zona da Mata,1.0,False,False,4.35825,2020-05-07,0.0,2020-05-07,0.0,estabilizando,NaN,NaN,6,city,RO,Rondônia,11,NaN,2020-08-14 02:17:21
6,NaN,1100015,Alta Floresta D'Oeste,2.0,1.0,estabilizando,87.2,0.0,0.066667,19,NaN,22945.0,0.005314,11005,Zona da Mata,2.0,False,False,8.71650,2020-05-08,0.0,2020-05-08,0.0,estabilizando,0.0,NaN,7,city,RO,Rondônia,11,NaN,2020-08-14 02:17:21
7,NaN,1100015,Alta Floresta D'Oeste,1.0,-1.0,estabilizando,0.0,0.0,0.066667,19,NaN,22945.0,0.005314,11005,Zona da Mata,1.0,False,False,4.35825,2020-05-09,0.0,2020-05-09,0.0,estabilizando,0.0,NaN,8,city,RO,Rondônia,11,NaN,2020-08-14 02:17:21
8,inf,1100015,Alta Floresta D'Oeste,1.0,0.0,estabilizando,0.0,0.0,0.066667,20,13.0,22945.0,0.005314,11005,Zona da Mata,1.0,False,False,4.35825,2020-05-10,0.0,2020-05-10,0.0,estabilizando,0.0,0.0,9,city,RO,Rondônia,11,193.0,2020-08-14 02:17:21
9,inf,1100015,Alta Floresta D'Oeste,1.0,0.0,estabilizando,0.0,0.0,0.000000,20,12.0,22945.0,0.005314,11005,Zona da Mata,1.0,False,False,4.35825,2020-05-11,0.0,2020-05-11,0.0,estabilizando,0.0,0.0,10,city,RO,Rondônia,11,205.0,2020-08-14 02:17:21


**First thing first**: Observar os valores em `active_cases`.

In [33]:
#percentual de casos ativos com valores infinitamente positivos
(df["active_cases"]==np.inf).sum() * 100/(df["active_cases"]).count()

7.8318541634390515

In [34]:
#percentual de casos ativos com valores infinitamente negativos
(df["active_cases"]==-np.inf).sum() * 100/(df["active_cases"]).count()

0.07328223250140785

**Cerca de 8% dos nossos dados tem casos ativos com valores infinitos.** Recortando-os e analisando, temos:

In [5]:
inf_active_cases = df[df["active_cases"].isin([np.inf, -np.inf])]
inf_active_cases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32146 entries, 8 to 525575
Data columns (total 33 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   active_cases                                   32146 non-null  float64
 1   city_id                                        32146 non-null  int64  
 2   city_name                                      32146 non-null  object 
 3   confirmed_cases                                32146 non-null  float64
 4   daily_cases                                    32146 non-null  float64
 5   daily_cases_growth                             32146 non-null  object 
 6   daily_cases_mavg_1mi                           27492 non-null  float64
 7   deaths                                         32146 non-null  float64
 8   deaths_mavg                                    32146 non-null  float64
 9   epidemiological_week                           32

In [6]:
inf_active_cases.head()

,active_cases,city_id,city_name,confirmed_cases,daily_cases,daily_cases_growth,daily_cases_mavg_1mi,deaths,deaths_mavg,epidemiological_week,estimated_cases,estimated_population_2019,expected_mortality,health_region_id,health_region_name,infectious_period_cases,is_last,is_repeated,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_updated,new_deaths,new_deaths_growth,new_deaths_mavg_1mi,notification_rate,order_for_place,place_type,state_id,state_name,state_num_id,total_estimated_cases,data_last_refreshed
8,inf,1100015,Alta Floresta D'Oeste,1.0,0.0,estabilizando,0.0,0.0,0.066667,20,13.0,22945.0,0.005314,11005,Zona da Mata,1.0,False,False,4.35825,2020-05-10,0.0,2020-05-10,0.0,estabilizando,0.0,0.0,9,city,RO,Rondônia,11,193.0,2020-08-14 02:17:21
9,inf,1100015,Alta Floresta D'Oeste,1.0,0.0,estabilizando,0.0,0.0,0.000000,20,12.0,22945.0,0.005314,11005,Zona da Mata,1.0,False,False,4.35825,2020-05-11,0.0,2020-05-11,0.0,estabilizando,0.0,0.0,10,city,RO,Rondônia,11,205.0,2020-08-14 02:17:21
19,inf,1100015,Alta Floresta D'Oeste,2.0,0.0,estabilizando,43.6,0.0,0.000000,21,12.0,22945.0,0.005314,11005,Zona da Mata,1.0,False,True,8.71650,2020-05-20,0.0,2020-05-21,0.0,estabilizando,0.0,0.0,20,city,RO,Rondônia,11,327.0,2020-08-14 02:17:21
22,inf,1100015,Alta Floresta D'Oeste,3.0,-1.0,estabilizando,87.2,0.0,0.066667,22,26.0,22945.0,0.005314,11005,Zona da Mata,2.0,False,False,13.07474,2020-05-24,0.0,2020-05-24,0.0,estabilizando,0.0,0.0,23,city,RO,Rondônia,11,378.0,2020-08-14 02:17:21
28,inf,1100015,Alta Floresta D'Oeste,3.0,0.0,estabilizando,-43.6,0.0,0.066667,22,26.0,22945.0,0.005314,11005,Zona da Mata,2.0,False,False,13.07474,2020-05-30,0.0,2020-05-30,0.0,estabilizando,0.0,0.0,29,city,RO,Rondônia,11,466.0,2020-08-14 02:17:21


**Como é calculado os casos ativos?** É uma razão entre casos de pessoas infectadas em um período por taxa de notificação: `df["infectious_period_cases"] / df["notification_rate"]`. O resultado de uma divisão pode dar infinito se o valor do denominador for zero. Sendo assim, vale investigar o valor de `notification_rate`.

In [7]:
(inf_active_cases["notification_rate"]==0).all()

True

Acima temos que para todos os casos ativos de valor infinito, a taxa de notificação é zero. Como é calculada a taxa de notificação? `notification_rate=lambda df: (df[cases] / df["estimated_cases"]).clip(0, 1)`
Ou seja, a taxa de notificação é dada pela razão entre o número de casos e o número de casos estimados. Sendo um valor que fica entra 0 e 1. Caso a divisão seja menor que zero, recebe o valor de zero. Caso seja maior que 1, recebe o valor de 1.

Ps.: `df[cases]` pode ser `df["confirmed_cases"]` se os casos e mortes considerados são acumulados ou `df["daily_cases"]`caso contrário.

In [8]:
(inf_active_cases["daily_cases"]==0).value_counts()

True     30230
False     1916
Name: daily_cases, dtype: int64

In [16]:
(inf_active_cases["daily_cases"]<0).value_counts()

False    31030
True      1116
Name: daily_cases, dtype: int64

In [40]:
(inf_active_cases["daily_cases"]==0 & inf_active_cases["is_repeated"]).value_counts()

True     30230
False     1916
dtype: int64

Pela documentação do brasil.io:
- `new_confirmed`(`daily_cases` na nossa base de dados): número de novos casos confirmados desde o último dia (note que caso `is_repeated` seja True, esse valor **sempre será 0** e que esse valor pode ser negativo caso a SES remaneje os casos desse município para outro).
- `is_repeated`: campo pré-computado que diz se as informações nesse registro foram publicadas pela Secretaria Estadual de Saúde no dia `date` ou se o dado é repetido do último dia em que o dado está disponível (igual ou anterior a `date`). Isso ocorre pois nem todas as secretarias publicam boletins todos os dias. Veja também o campo `last_available_date`.

In [26]:
#percentual de casos diários menor ou igual a zero
(inf_active_cases["daily_cases"]<=0).sum() * 100/(inf_active_cases["daily_cases"]).count()

97.51135444534312

In [9]:
(inf_active_cases["confirmed_cases"]==0).value_counts()

False    32047
True        99
Name: confirmed_cases, dtype: int64

In [17]:
(inf_active_cases["confirmed_cases"]<0).value_counts()

False    32146
Name: confirmed_cases, dtype: int64

In [27]:
#percentual de casos confirmados menor ou igual a zero
(inf_active_cases["confirmed_cases"]<=0).sum() * 100/(inf_active_cases["confirmed_cases"]).count()

0.30796988738878867

Percebemos que 97.5% dos **casos novos diários são estimados como zero ou menor que zero**, mas apenas 0.3% nos acumulados. Podemos dizer que parte do nosso problema pode estar em casos novos diários estarem zerados ou negativos (o que pela regra do `clip(0,1)` levaria o valor a zero).
Agora, olhando o denominador, vamos ver como são os casos estimados (`estimated_cases`).

In [10]:
(inf_active_cases["estimated_cases"]).describe()

count    32146.000000
mean       161.314845
std        636.179029
min          5.000000
25%         14.000000
50%         32.000000
75%         99.000000
max      16435.000000
Name: estimated_cases, dtype: float64

Boas notícias. Nada anormal aqui. 

**Resumo até agora:**
- Todas as taxas de notificação de casos diários infinitos estão zeradas.
- Uma boa parte dos casos diários também estão zerados.
- Os casos estimados não possuem valores zerados ou infinitos.

**Para onde ir:**

Podemos observar se sempre o caso não é acumulado o valor é zerado e o mesmo para não acumulado. Isso provavelmente nos guia em uma parte da solução.
Outro ponto é observar se o dia anterior possui `is_repeated` igual a False. Uma vez que se for igual a True o valor sempre será zerado, poderiamos considerar o valor anterior mais recente como o atual.